<a href="https://colab.research.google.com/github/aparna2004/KLA-Unhack/blob/main/Milestone3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image text detection with slightly titlted or changed background and labled car data (using ocr)

###Installing Required Packages

####**Installing easyocr**
- Dependenciecs already available in colab

####**Downgrading to python-bidi 0.4.2**
- Because bidi.algorithm needed by easyocr not supported in higher *versions*

In [ ]:
!pip install easyocr --no-deps
!pip install python-bidi==0.4.2

### Speed limit detection using OCR
####**Libraries used**
- pandas -> data frame
- glob(function in python lib) -> To match image file paths
- easyocr -> Text detection using optical character recognition

####**Input**
- Images containing speed limit boards which is slightly rotated or has a different background
with vehicles labeled along with speed of vehicles

####**Output**
- CSV file with four columns: Image Name, Speed limit, No. of Vehicles, No, of Overspeeding vehicles

####**Function**
- Detects text from images
- Takes number from a given array(all speed limits specified in templates)only numbers
- Remaining text added as other text for extraction of vehicle data
- Calculates number of vehicles from a given list of vehicles
- Calculates number of vehicles overspeeding by checking for speed specified after vehicle name
- Handles case when number is 0
- Returns sorted data(according to order of input) ignoring confidence and bounding box


In [20]:
import pandas as pd
from glob import glob
import easyocr
import re

def speed_limit_detection(imgs, reader):
    speed_limits = {'10', '15', '20', '25', '40', '45', '50', '55', '70', '75', '80', '85'}
    vehicles = ['Car', 'Bicycle', 'Truck']
    dfs = []

    def count_vehicles(text):
        return sum(len(re.findall(vehicle, text, re.IGNORECASE)) for vehicle in vehicles)

    def extract_vehicle_speeds(text):
        return [(vehicle, int(speed)) for vehicle, speed in re.findall(r'(Car|Bicycle|Truck)\s?\d*:\s?(\d+)', text, re.IGNORECASE)]

    for img in imgs:
        detected_text = reader.readtext(img)
        img_id = img.split('/')[-1]

        if not detected_text:
            dfs.append({'Image name': img_id, 'Speed limit': 'None', 'No. of Vehicles': 'none', 'No. of Overspeeding Vehicles': 'None'})
            continue

        filtered_text = [entry for entry in detected_text if entry[1] in speed_limits]
        other_texts = ' '.join(entry[1] for entry in detected_text if entry[1] not in speed_limits)

        if filtered_text:
            speed_limit = filtered_text[0][1]
            speed_limit_value = int(speed_limit)
        else:
            speed_limit = 'None'
            speed_limit_value = float('inf')

        no_of_vehicles = count_vehicles(other_texts)
        vehicle_speeds = extract_vehicle_speeds(other_texts)
        no_of_overspeeding_vehicles = sum(1 for _, speed in vehicle_speeds if speed > speed_limit_value)

        dfs.append({
            'Image name': img_id,
            'Speed limit': speed_limit,
            'No. of Vehicles': no_of_vehicles,
            'No. of Overspeeding Vehicles': no_of_overspeeding_vehicles
        })

    df = pd.DataFrame(dfs)
    df['No. of Overspeeding Vehicles'] = df['No. of Overspeeding Vehicles'].replace(0, 'None')

    return df

def main():
    reader = easyocr.Reader(['en'], gpu=True)
    images = glob('../content/sample_data/Milestone3/*')

    df = speed_limit_detection(images, reader)

    df_grouped = df.groupby('Image name').first().reset_index()

    df_grouped.to_csv('M3_Output.csv', index=False)
    print(df_grouped)

if __name__ == "__main__":
    main()


     Image name Speed limit  No. of Vehicles No. of Overspeeding Vehicles
0  Picture1.jpg          20                2                            1
1  Picture2.jpg          80                5                            2
2  Picture3.jpg          15                3                            2
3  Picture4.jpg          40                2                            1
4  Picture5.jpg          25                3                         None
5  Picture6.jpg          85                3                            2
6  Picture7.jpg        None                4                         None
7  Picture8.jpg          85                0                         None
